## run this cell before starting spark cluster connection (corrects module error)

In [50]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [51]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [52]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data

In [53]:
df = spark.read.json("/user/rucio01/traces/traces.2018-04*")

### Create temporary table view

In [54]:
# body_schema = spark.read.json(df_loadAvg.rdd.map(lambda row: row.body)).schema
df.createOrReplaceTempView("traces")

### set Pandas Options

In [55]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 1900)
pd.set_option('display.max_columns', None)

### query user and official data (excluding pilot and None-existing Datasets)

In [56]:
df_ps = spark.sql("SELECT *\
                   FROM traces \
                   WHERE eventType=='download' AND dataset != 'None' AND dataset != '' AND dataset != '[null]' AND account != 'pilot'")\

In [57]:
#df_ps.describe().toPandas()

### query only official data (excluding pilot and None-existing Datasets)

In [58]:
df_ps_official = spark.sql("SELECT *\
                   FROM traces \
                   WHERE eventType=='download' AND dataset != 'None' AND dataset != '' AND dataset != '[null]' AND account != 'pilot' AND (dataset LIKE 'data%' OR dataset LIKE 'mc%')")\

In [59]:
#df_ps_official.describe().toPandas()

### drop columns that do not contain any values (or very little values, like 'name') based on previous describe()

In [60]:
df_ps = df_ps.drop('appid', 'catStart', 'datasetName', 'dataset_scope', 'duid', 
                   'guid', 'ip', 'localSite', 'pq', 'relativeStart', 'stateReason', 
                   'suspicious', 'taskid', 'timeEnd', 'timeStart', 'url', 'usr', 
                   'usrdn', 'validateStart', 'version', 'name')

In [61]:
df_ps_official = df_ps_official.drop('appid', 'catStart', 'datasetName', 'dataset_scope', 'duid', 
                   'guid', 'ip', 'localSite', 'pq', 'relativeStart', 'stateReason', 
                   'suspicious', 'taskid', 'timeEnd', 'timeStart', 'url', 'usr', 
                   'usrdn', 'validateStart', 'version', 'name')

In [62]:
#df_ps = df_ps.dropna()

### Split dataset column according to schema for official data

In [63]:
split_col = split(df_ps_official['dataset'], "\.")
df_ps_official = df_ps_official.withColumn('project', split_col.getItem(0))
df_ps_official = df_ps_official.withColumn('run_number', split_col.getItem(1))
df_ps_official = df_ps_official.withColumn('stream_name', split_col.getItem(2))
df_ps_official = df_ps_official.withColumn('prod_step', split_col.getItem(3))
df_ps_official = df_ps_official.withColumn('datatype', split_col.getItem(4))
df_ps_official = df_ps_official.withColumn('dataset_version', split_col.getItem(5))

In [64]:
#df_ps_official.describe().toPandas()

### Count dataset hits and join hits column to tables

In [67]:
df_ps_counts = spark.sql("SELECT filename AS file_counts, count(*) as hits \
                          FROM traces \
                          WHERE eventType=='download' AND dataset != 'None' AND dataset != '' AND dataset != '[null]' AND account != 'pilot'\
                          GROUP BY  filename \
                          ORDER BY 2 DESC")

In [68]:
df_ps_counts.createOrReplaceTempView("counts")
df_ps.createOrReplaceTempView("df_ps")
df_ps_official.createOrReplaceTempView("df_ps_official")

In [69]:
df_ps_merged = spark.sql("SELECT * \
                          FROM df_ps \
                          INNER JOIN counts ON df_ps.filename = counts.file_counts")

In [70]:
df_ps_merged_official = spark.sql("SELECT * \
                          FROM df_ps_official \
                          INNER JOIN counts ON df_ps_official.filename = counts.file_counts")

In [71]:
df_ps_merged = df_ps_merged.drop('file_counts')
df_ps_merged_official = df_ps_merged_official.drop('file_counts')

In [72]:
# refresh merged view so that all columns are in it
df_ps_merged.createOrReplaceTempView("merged")
df_ps_merged_official.createOrReplaceTempView("merged_official")

In [73]:
#df_ps_merged.describe().toPandas()

In [74]:
#df_ps_merged_official.describe().toPandas()

## cast filesize and run_number to double

In [75]:
df_ps_final = df_ps_merged.withColumn("filesize_double",df_ps_merged.filesize.cast(DoubleType()))
df_ps_final_official = df_ps_merged_official.withColumn("filesize_double",df_ps_merged_official.filesize.cast(DoubleType()))

In [76]:
#df_ps_final.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps_final.columns]).show()

In [77]:
df_ps_final = df_ps_final.drop('filesize')
df_ps_final_official = df_ps_final_official.drop('filesize')
df_ps_final.createOrReplaceTempView("final")
df_ps_final.createOrReplaceTempView("final_official")

## analyse

In [78]:
df_hits_cleaned = spark.sql("SELECT filename, hits, count(*)\
                     FROM final \
                     WHERE clientState != 'ALREADY_DONE' AND account == 'tnitta'\
                     GROUP BY filename, hits \
                     ORDER BY 2 DESC")
df_hits_cleaned_pd = df_hits_cleaned.toPandas()

In [79]:
df_hits_cleaned_pd

filename  \
0      user.tnitta.13709374._000841.DAOD_TRUTH1.363600.root                                                                       
1      user.tnitta.13709374._000844.DAOD_TRUTH1.363600.root                                                                       
2      user.tnitta.13709374._000848.DAOD_TRUTH1.363600.root                                                                       
3      user.tnitta.13709374._000347.DAOD_TRUTH1.363600.root                                                                       
4      user.tnitta.13709374._000332.DAOD_TRUTH1.363600.root                                                                       
5      user.tnitta.13709374._000686.DAOD_TRUTH1.363600.root                                                                       
6      user.tnitta.13709374._000589.DAOD_TRUTH1.363600.root                                                                       
7      user.tnitta.13709374._000852.DAOD_TRUTH1.363600.root                                                                       
8      user.tnitta.13709374._000334.DAOD_TRUTH1.363600.root                                                                       
9      user.tnitta.13709374._000846.DAOD_TRUTH1.363600.root                                                                       
10     user.tnitta.13709374._000839.DAOD_TRUTH1.363600.root                                                                       
11     user.tnitta.13709374._000851.DAOD_TRUTH1.363600.root                                                                       
12     user.tnitta.13709374._000842.DAOD_TRUTH1.363600.root                                                                       
13     user.tnitta.13709374._000348.DAOD_TRUTH1.363600.root                                                                       
14     user.tnitta.13709374._000853.DAOD_TRUTH1.363600.root                                                                       
15     user.tnitta.13709374._000355.DAOD_TRUTH1.363600.root                                                                       
16     user.tnitta.13709374._000845.DAOD_TRUTH1.363600.root                                                                       
17     user.tnitta.13709374._000838.DAOD_TRUTH1.363600.root                                                                       
18     user.tnitta.13709374._000850.DAOD_TRUTH1.363600.root                                                                       
19     user.tnitta.13709374._000312.DAOD_TRUTH1.363600.root                                                                       
20     user.tnitta.13709374._000317.DAOD_TRUTH1.363600.root                                                                       
21     user.tnitta.13709374._000847.DAOD_TRUTH1.363600.root                                                                       
22     user.tnitta.13709374._000321.DAOD_TRUTH1.363600.root                                                                       
23     user.tnitta.13709374._000840.DAOD_TRUTH1.363600.root                                                                       
24     user.tnitta.13709374._000320.DAOD_TRUTH1.363600.root                                                                       
25     user.tnitta.13709374._000843.DAOD_TRUTH1.363600.root                                                                       
26     user.tnitta.13709374._000351.DAOD_TRUTH1.363600.root                                                                       
27     user.tnitta.13709374._000407.DAOD_TRUTH1.363600.root                                                                       
28     user.tnitta.13709374._000375.DAOD_TRUTH1.363600.root                                                                       
29     user.tnitta.13709374._000571.DAOD_TRUTH1.363600.root                                                                       
30     user.tnitta.13709374._000439.DAOD_TRUTH1.363600.roo